In [1]:
# ------------------------------------------------------------------------
#
# TITLE - fit_dens_gse.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Fit density profiles to GS/E Gaia/APOGEE halo data
'''

__author__ = "James Lane"

In [2]:
### Imports

# Basic
import os, sys, copy, dill as pickle
import numpy as np

# Matplotlib and plotting 
import matplotlib.pyplot as plt

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

A new version of galpy (1.9.0) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!
/home/lane/software/galpy/galpy/util/bovy_plot.py:7: FutureWarning: galpy.util.bovy_plot is being deprecated in favor of galpy.util.plot; functions inside of this module have also changed name, but all functions still exist; please switch to the new import and new function names, because the old import and function names will be removed in v1.9
  warnings.warn(



### Keywords, Pathing, Loading, Data Preparation

In [3]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
feh_range_fit = copy.deepcopy(feh_range_gse) # Need to choose here

## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max],ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid
# apogee_SF,apogee_effSF_grid_inclArea,apogee_effSF_grid_inclArea_Jac = other_stuff

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.9]
# dfs = putil.load_distribution_functions(df_filename, betas)

# ## Load the APOGEE field information, can also similarly load 
# ## '...apogee_field_glons.npy', '...apogee_field_glats.npy', 
# ## '...apogee_field_location_ids.npy'
# apogee_fields = np.load(ga_dir+'apogee_fields.npy')

Loading config file from /home/lane/projects/ges-mass/notebooks/config.txt

Loading APOGEE sel. func. from /epsen_data/scr/lane/projects/ges-mass/data/gaia_apogee/apogee_dr16_l33_gaia_edr3/apogee_SF.dat
/home/lane/software/galpy/galpy/util/bovy_coords.py:7: FutureWarning: galpy.util.bovy_coords is being deprecated in favor of galpy.util.coords; all functions in there are the same; please switch to the new import, because the old import will be removed in v1.9
  warnings.warn(


Loading APOGEE eff. sel. func. from /epsen_data/scr/lane/projects/ges-mass/data/gaia_apogee/apogee_dr16_l33_gaia_edr3/apogee_effSF_grid_inclArea_-2.0_feh_-0.6.dat

Loading APOGEE eff. sel. func. mask from /epsen_data/scr/lane/projects/ges-mass/data/gaia_apogee/apogee_dr16_l33_gaia_edr3/apogee_effSF_grid_mask.npy

Loading cleaned kinematics from /epsen_data/scr/lane/projects/ges-mass/data/gaia_apogee_processed/apogee_dr16_l33_gaia_edr3/clean_kinematics_sampled.npy

Loading isochrone grid from /epsen_data/scr/lane

### Global Parameters

In [4]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = False
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 50
nit = int(2e3)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 5000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

# Fits to GS/E halo data. Use -2 < [Fe/H] < -0.6

## Fits to data with e-Lz selection, GS/E [Fe/H] range

In [5]:
densfunc = pdens.triaxial_single_angle_zvecpa
selec = ['eLz',]
fit_type = 'gse'
version = 'hmask_50w_2e3n'

hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
print(hf.n_star)

Loading APOGEE kin. eff. sel. grid from /epsen_data/scr/lane/projects/ges-mass/data/ksf/apogee_dr16_l33_gaia_edr3/v1_alpha35_rc30/v1_beta_03_08_5050mix/kSF_grid_linear_eLz.dat
Using informed init
init_type: uninformed
densfunc_source: None
163


### Triaxial Single power law

In [6]:
def usr_log_prior(densfunc,params):
    return 0

densfunc = pdens.triaxial_single_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_angle_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: uninformed
densfunc_source: None
Doing maximum likelihood to find starting point
MLE successful
MLE result: [2.82216591 0.95867932 0.44003705 0.27100062 0.99952416 0.44372288]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 39.313987121678075
mean nit/tau: 12.718119850130785
min nit/tau: 11.702333251035002
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.25,0.5]
total max nit/tau 11.702333251035002
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:04:51.280111
---

sampled 1000/2000 per walker
mean tau: 64.5624459617798
mean nit/tau: 15.488880340623837
min nit/tau: 13.503036785569323
[min,max] delta tau: [-0.45277841225997423,-0.3361927411612028]
[min,max] acceptance fraction: [0.27,0.42]
total max nit/tau 13.503036785569323
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:03:12.530050
---

sampled 1

### Triaxial exponential cutoff

In [7]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_single_cutoff_zvecpa
    if params[1] < 2.: return -np.inf
    if params[1] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_single_cutoff_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_cutoff_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/gse/eLz/-2.0_feh_-0.6/triaxial_single_angle_zvecpa/hmask_50w_2e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [-1.10078393  4.37988149  0.62249154  0.32772265  0.22227466  0.99776551
  0.53769722]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 41.78567870579052
mean nit/tau: 11.965822154534292
min nit/tau: 10.293055268703718
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.24,0.46]
total max nit/tau 10.293055268703718
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:45.166724
---

sampled 1000/2000 per walker
mean tau: 74.40440759342673
mean nit/tau: 13.440064000836763
min nit/tau: 12.039276997638478
[min,max] delta tau: [-0.5184710856424786,-0.4

### Triaxial broken power law

In [8]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] < 2.: return -np.inf
    if params[2] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_broken_angle_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/gse/eLz/-2.0_feh_-0.6/triaxial_single_angle_zvecpa/hmask_50w_2e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 1.71949144  5.88299174 18.46310161  0.67627727  0.38545443  0.16870651
  0.99844276  0.52731872]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 41.62965391985982
mean nit/tau: 12.010669148548224
min nit/tau: 10.980534847875967
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.24,0.47]
total max nit/tau 10.980534847875967
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:29.735119
---

sampled 1000/2000 per walker
mean tau: 71.74396031318838
mean nit/tau: 13.938455524822405
min nit/tau: 12.205180764022025
[min,max] delta tau: [-0.4568135311

### Triaxial double broken power law

In [9]:
def usr_log_prior(densfunc,params):
    # return 0
    assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_double_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] > 10.: return -np.inf
    #if params[3] < 2.: return -np.inf
    if params[3] > 55.: return -np.inf
    #if params[4] < 2.: return -np.inf
    if params[4] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_double_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_double_broken_angle_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_broken_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/gse/eLz/-2.0_feh_-0.6/triaxial_broken_angle_zvecpa/hmask_50w_2e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 1.72303107  5.4987419   9.9996038  17.92760614 31.8195017   0.6928419
  0.38557378  0.13030407  0.99829919  0.53972455]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 47.615466641468245
mean nit/tau: 10.500789664939473
min nit/tau: 8.390447893118404
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.16,0.39]
total max nit/tau 8.390447893118404
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:17.629617
---

sampled 1000/2000 per walker
mean tau: 88.35200325828052
mean nit/tau: 11.318362494585294
min nit/tau: 8.518292213760565
[min,ma

## Fits to data with AD selection, GS/E [Fe/H] range

In [10]:
selec = ['AD',]
densfunc = pdens.triaxial_single_angle_zvecpa
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
print(hf.n_star)

Loading APOGEE kin. eff. sel. grid from /epsen_data/scr/lane/projects/ges-mass/data/ksf/apogee_dr16_l33_gaia_edr3/v1_alpha35_rc30/v1_beta_03_08_5050mix/kSF_grid_linear_AD.dat
Using informed init
init_type: uninformed
densfunc_source: None
77


### Triaxial single power law

In [11]:
def usr_log_prior(densfunc,params):
    return 0

densfunc = pdens.triaxial_single_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_angle_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: uninformed
densfunc_source: None
Doing maximum likelihood to find starting point
MLE successful
MLE result: [2.16374816 0.53519987 0.26811571 0.35742117 0.96400095 0.5322309 ]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 41.278743695098484
mean nit/tau: 12.112771737754484
min nit/tau: 10.223386942491953
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.28,0.45]
total max nit/tau 10.223386942491953
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:04:54.218732
---

sampled 1000/2000 per walker
mean tau: 64.67154716664851
mean nit/tau: 15.462750526489735
min nit/tau: 13.539509598346651
[min,max] delta tau: [-0.44799072220863906,-0.30391456101817726]
[min,max] acceptance fraction: [0.27,0.4]
total max nit/tau 13.539509598346651
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:03:11.242584
---

sampled

### Triaxial exponential cutoff

In [12]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_single_cutoff_zvecpa
    if params[1] < 2.: return -np.inf
    if params[1] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_single_cutoff_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_cutoff_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/gse/AD/-2.0_feh_-0.6/triaxial_single_angle_zvecpa/hmask_50w_2e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [-1.25091115  7.44920797  0.44222335  0.2244615   0.39322967  0.96831724
  0.56201604]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 45.614452388185946
mean nit/tau: 10.961438180709125
min nit/tau: 9.931332853485223
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.17,0.42]
total max nit/tau 9.931332853485223
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:46.627673
---

sampled 1000/2000 per walker
mean tau: 80.93376124400564
mean nit/tau: 12.3557831074367
min nit/tau: 11.524168337843683
[min,max] delta tau: [-0.49086816233920166,-0.3910

### Triaxial broken power law

In [13]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] < 2.: return -np.inf
    if params[2] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_broken_angle_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/gse/AD/-2.0_feh_-0.6/triaxial_single_angle_zvecpa/hmask_50w_2e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 0.42287938  3.61303331 18.45854633  0.57417782  0.29524345  0.39859844
  0.96936613  0.5445506 ]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 43.967982708309094
mean nit/tau: 11.371911313673023
min nit/tau: 9.931680813397275
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.17,0.4]
total max nit/tau 9.931680813397275
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:11.455753
---

sampled 1000/2000 per walker
mean tau: 82.43417118112444
mean nit/tau: 12.130891663880492
min nit/tau: 11.416155194726947
[min,max] delta tau: [-0.5350327617131

In [14]:
# nit = int(2e4)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

# def usr_log_prior(densfunc,params):
#     # return 0
#     assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa_inv
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] > 1.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_broken_angle_zvecpa_inv
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# # densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
# # hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# # fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# nit = int(1e6)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

### Triaxial double broken power law

In [15]:
def usr_log_prior(densfunc,params):
    # return 0
    assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_double_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] > 10.: return -np.inf
    #if params[3] < 2.: return -np.inf
    if params[3] > 55.: return -np.inf
    #if params[4] < 2.: return -np.inf
    if params[4] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_double_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_double_broken_angle_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_broken_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/gse/AD/-2.0_feh_-0.6/triaxial_broken_angle_zvecpa/hmask_50w_2e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 0.54792529  3.57291307  3.82880847 24.39960843 38.65802922  0.43559701
  0.21818763  0.382107    0.96476164  0.54771279]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 49.25972409362467
mean nit/tau: 10.150280156861687
min nit/tau: 8.819139914992332
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.15,0.35]
total max nit/tau 8.819139914992332
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:03.282437
---

sampled 1000/2000 per walker
mean tau: 90.76409195860995
mean nit/tau: 11.017572901583348
min nit/tau: 9.39151471223664
[min,max]

## Fits to data with JRLz selection, GS/E [Fe/H] range

In [16]:
selec = ['JRLz',]
densfunc = pdens.triaxial_single_angle_zvecpa
hf = pmass.HaloFit(densfunc=densfunc,
                   selec=selec,
                   fit_type=fit_type,
                   feh_range=feh_range_fit,
                   version=version,
                   **hf_kwargs)
print(hf.n_star)

Loading APOGEE kin. eff. sel. grid from /epsen_data/scr/lane/projects/ges-mass/data/ksf/apogee_dr16_l33_gaia_edr3/v1_alpha35_rc30/v1_beta_03_08_5050mix/kSF_grid_linear_JRLz.dat
Using informed init
init_type: uninformed
densfunc_source: None
319


### Triaxial single power law

In [17]:
def usr_log_prior(densfunc,params):
    return 0

densfunc = pdens.triaxial_single_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_angle_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: uninformed
densfunc_source: None
Doing maximum likelihood to find starting point
MLE successful
MLE result: [2.61998417 0.68103516 0.61519402 0.05424351 0.5        0.55736609]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 40.245321410016196
mean nit/tau: 12.423804369855542
min nit/tau: 8.752198939864288
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.35,0.51]
total max nit/tau 8.752198939864288
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:15.873177
---

sampled 1000/2000 per walker
mean tau: 70.49768862697248
mean nit/tau: 14.1848622199707
min nit/tau: 9.817455660007013
[min,max] delta tau: [-0.5535098184627584,-0.2829356745228505]
[min,max] acceptance fraction: [0.34,0.45]
total max nit/tau 9.817455660007013
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:03:26.150134
---

sampled 1500/2

### Triaxial exponential cutoff

In [18]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_single_cutoff_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_single_cutoff_zvecpa
    if params[1] < 2.: return -np.inf
    if params[1] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_single_cutoff_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_single_cutoff_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/gse/JRLz/-2.0_feh_-0.6/triaxial_single_angle_zvecpa/hmask_50w_2e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 1.77832933 24.53761951  0.57146797  0.65547035  0.48908996  0.93176258
  0.59643922]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 39.81219904695547
mean nit/tau: 12.5589646382077
min nit/tau: 9.772875935637737
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.24,0.46]
total max nit/tau 9.772875935637737
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:53.814044
---

sampled 1000/2000 per walker
mean tau: 69.42943473383112
mean nit/tau: 14.403113086454766
min nit/tau: 11.212895546831316
[min,max] delta tau: [-0.5038651651536434,-0.3406

### Triaxial broken power law

In [19]:
def usr_log_prior(densfunc,params):
    assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] < 2.: return -np.inf
    if params[2] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_broken_angle_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_single_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/gse/JRLz/-2.0_feh_-0.6/triaxial_single_angle_zvecpa/hmask_50w_2e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 2.44576887  2.73292324 17.45608338  0.58936531  0.65633582  0.49957115
  0.89611942  0.59478436]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 44.85541284427628
mean nit/tau: 11.146926720657792
min nit/tau: 8.55771126908531
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.2,0.43]
total max nit/tau 8.55771126908531
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:18.258835
---

sampled 1000/2000 per walker
mean tau: 81.60305873166374
mean nit/tau: 12.254442609661377
min nit/tau: 9.614526953822617
[min,max] delta tau: [-0.490826881355651

In [20]:
# nit = int(2e4)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

# def usr_log_prior(densfunc,params):
#     # return 0
#     assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa_inv
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] > 1.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_broken_angle_zvecpa_inv
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# # densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
# # hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# # fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# nit = int(1e6)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

### Triaxial double broken power law

In [21]:
def usr_log_prior(densfunc,params):
    # return 0
    assert densfunc is pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk\
        or densfunc is pdens.triaxial_double_broken_angle_zvecpa
    if params[0] > 10.: return -np.inf
    if params[1] > 10.: return -np.inf
    if params[2] > 10.: return -np.inf
    #if params[3] < 2.: return -np.inf
    if params[3] > 55.: return -np.inf
    #if params[4] < 2.: return -np.inf
    if params[4] > 55.: return -np.inf
    return 0

densfunc = pdens.triaxial_double_broken_angle_zvecpa
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

densfunc = pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk
hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
pmass.fit(hf, nprocs=nprocs, **fit_kwargs)

Setting densfunc to: triaxial_double_broken_angle_zvecpa
Version is: hmask_50w_2e3n/
Using init_type: ML
init_type: ML
densfunc_source: triaxial_broken_angle_zvecpa
source_fit_data_dir: /epsen_data/scr/lane/projects/ges-mass/data/fitting/data/gse/JRLz/-2.0_feh_-0.6/triaxial_broken_angle_zvecpa/hmask_50w_2e3n/
Doing maximum likelihood to find starting point
MLE successful
MLE result: [ 2.17568607  2.62972301  2.71837666 11.85168945 19.86417202  0.57914176
  0.64674013  0.503155    0.91225716  0.59495158]
Running MCMC with 10 processers
Generating MCMC samples...
sampled 500/2000 per walker
mean tau: 48.003072082771006
mean nit/tau: 10.416000024703777
min nit/tau: 8.983285118587089
[min,max] delta tau: [inf,inf]
[min,max] acceptance fraction: [0.15,0.38]
total max nit/tau 8.983285118587089
(nit/tau)/max[(nit/tau)] 1.0
estimated time remaining: 0:05:14.702502
---

sampled 1000/2000 per walker
mean tau: 90.85261329078023
mean nit/tau: 11.006838039973921
min nit/tau: 8.783264841294521
[min,

In [ ]:
sys.exit()

## Triaxial broken power law with inverted scale

In [ ]:
# nit = int(2e4)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

# def usr_log_prior(densfunc,params):
#     # return 0
#     assert densfunc is pdens.triaxial_broken_angle_zvecpa_plusexpdisk\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa\
#         or densfunc is pdens.triaxial_broken_angle_zvecpa_inv
#     if params[0] > 10.: return -np.inf
#     if params[1] > 10.: return -np.inf
#     if params[2] > 1.: return -np.inf
#     return 0

# densfunc = pdens.triaxial_broken_angle_zvecpa_inv
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# # densfunc = pdens.triaxial_broken_angle_zvecpa_plusexpdisk
# # hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# # fit_data(hf, nprocs=nprocs, **fit_data_kwargs)

# nit = int(1e6)

# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=usr_log_prior, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

## Fit to data with e-Lz selection, GS/E [Fe/H] range. Including Log priors

In [ ]:
# feh_range_fit = copy.deepcopy(feh_range_gse)
# selec = ['eLz',]
# fit_type = 'gse'
# version = 'logprior_test'
# nit = int(2e4)

# densfunc = pdens.triaxial_single_cutoff_zvecpa
# hf = pmass.HaloFit(allstar=allstar_nomask, orbs=orbs_nomask, densfunc=densfunc,
#     nwalkers=nwalkers, nit=nit, ncut=ncut, selec=selec, 
#     usr_log_prior=None, n_mass=n_mass, effsel=apof, 
#     effsel_grid=effsel_grid, effsel_mask=apogee_effSF_mask, 
#     iso_filename=iso_grid_filename, int_r_range=int_r_range, 
#     feh_range=feh_range_gse, logg_range=logg_range, jkmins=jkmins, init=None, 
#     init_type='ML', fit_dir=fit_dir, gap_dir=gap_dir, ksf_dir=ksf_dir, 
#     fit_type=fit_type, version=version, verbose=verbose, ro=ro, vo=vo, zo=zo)

### Triaxial exponential cutoff

In [ ]:
# def usr_log_prior(densfunc,params):
#     a,b = 1.,55.
#     assert densfunc.__name__ == 'triaxial_single_cutoff_zvecpa'
#     prior = scipy.stats.loguniform.pdf(params[1],a,b)
#     if prior == 0:
#         return -np.inf
#     else:
#         return np.log(prior)

# densfunc = pdens.triaxial_single_cutoff_zvecpa
# hf.set_densfunc(densfunc,usr_log_prior=usr_log_prior)
# fit_data(hf, force_fit=force_fit, mle_init=mle_init, just_mle=just_mle,
#          return_walkers=return_walkers, mass_int_type=mass_int_type, 
#          batch_masses=batch_masses, make_ml_aic_bic=make_ml_aic_bic, 
#          calculate_masses=calculate_masses)